In [290]:
# takes a file, looks for lines that start with #. MediAustritt

import re
name_line = re.compile(r'(?P<id>\d+)\. MediAustritt - Markenname: ')
ingredient_line = re.compile(r'(?P<id>\d+)\. MediAustritt - Wirkstoff: ')
atc_line = re.compile(r'(?P<id>\d+)\. MediAustritt - ATC Code: ')

brand_name_r = re.compile(r'(?P<brand_name>.+)\(')
form_r = re.compile(r'\((\D+)( \d.*)?\)')
dose_r = re.compile(r'\) (\d.+)')


def annotate_drugs(inpath,outpath):
    with open(inpath,'r') as ip, open(outpath,'w') as op:
        manual = ''
        for line in ip:
            name = name_line.match(line)
            ingredient = ingredient_line.match(line)
            atc = atc_line.match(line)
            
            if name:
                line_id = name.group('id')
                nitty_gritty = line[name.end():]
                try:
                    brand_name = brand_name_r.match(nitty_gritty).group('brand_name').strip()
                except:
                    m = re.search("\d", nitty_gritty)
                    if m:
                        name_form = nitty_gritty[0:m.start()].strip()
                        dose = nitty_gritty[m.start():].strip()
                    else:
                        name_form = nitty_gritty
                    
                    parts = name_form.split(' ')
                    
                    if len(parts) > 1:
                        brand_name = ' '.join(name_form.split(' ')[:-1]).strip()
                        form = name_form.split(' ')[-1].strip()
                    else:
                        brand_name = name_form.strip()                  

                try:
                    form_dose = form_r.findall(nitty_gritty)[0]
                    form = form_dose[0].strip()
                    dose = form_dose[1].lstrip()
                except:
                    # already taken care of in the previous try-except block
                    pass
                
                if not dose:
                    dose_candidate = dose_r.findall(nitty_gritty)
                    if dose_candidate:
                        dose = dose_candidate[0].lstrip()
                
                brand_name_xml = '''<drug id='dr{}' type='brand_name'>{}</drug>'''.format(line_id,brand_name)
                xml_line = line.replace(brand_name,brand_name_xml)
                
                if form:
                    form_xml = '''<drug id='dr{}' type='form'>{}</drug>'''.format(line_id,form)
                    xml_line = xml_line.replace(form,form_xml)

                if dose:
                    dose_xml = '''<drug id='dr{}' type='dosage'>{}</drug>'''.format(line_id,dose)
                    xml_line = xml_line.replace(dose,dose_xml)
                op.write(xml_line)
                
                last_id = line_id
                    
            elif ingredient:
                line_id = ingredient.group('id')
                nitty_gritty = line[ingredient.end():]
                
                # we know form from previous line
                if last_id == line_id:

                    i = nitty_gritty.find(form)
                    if i > 0:
                        ingredient = nitty_gritty[0:i].strip()
                        dose = nitty_gritty[i+len(form)+1:].strip()
                    else:
                        # give up and annotate all
                        ingredient = nitty_gritty.strip()
                        manual = ' check_manually="True"'
                else:
                    m = re.search("\d", nitty_gritty)
                    if m:
                        ingredient = nitty_gritty[0:m.start()].strip()
                        dose = nitty_gritty[m.start():].strip()
                    else:
                        ingredient = nitty_gritty
                
                ingredient_xml = '''<drug id='dr{}' type='ingredient'{}>{}</drug>'''.format(line_id,manual,ingredient)
                
                # switch set in ingredient part if we fail to take line apart
                manual = ''
                xml_line = line.replace(ingredient,ingredient_xml)
                
                form_xml = '''<drug id='dr{}' type='form'>{}</drug>'''.format(line_id,form)
                xml_line = xml_line.replace(form,form_xml)
                
                if dose:
                    dose_xml = '''<drug id='dr{}' type='dosage'>{}</drug>'''.format(line_id,dose)
                    xml_line = xml_line.replace(dose,dose_xml)
                    
                op.write(xml_line)
                
            elif atc:
                line_id = atc.group('id')
                atc = line[atc.span()[1]:].strip()
                
                atc_xml = '''<drug id='dr{}' type='atc'>{}</drug>'''.format(line_id,atc) 
                xml_line = line.replace(atc,atc_xml)
                op.write(xml_line)

            else:
                op.write(line)

Folsäure Tabl 5mg
Tabl
Acetylsalicylsäure Filmtabl 100mg
Filmtabl
Amiodaron Tabl 200mg
Tabl
Pipamperon Tabl 40mg
Tabl
Esomeprazol Tabl 40mg
Tabl
Kalium chlorid Ret Drag
Ret Drag
Magnesium Gran 10mmol
Gran
Olmesartanmedoxomil Filmtabl 20mg
Filmtabl
Tamsulosin Ret Tabl 0.4mg
Ret Tabl
Prednison Tabl 50mg
Tabl
Risperidon Filmtabl 1mg
Filmtabl
Colecalciferol Tropfen 100U.I./Tr
Tropfen
Lercanidipin Filmtabl 20mg
Filmtabl
Dalteparin Inj Lös 18000U.I.
Inj Lös
Methotrexat Inj Lös 20mg
Inj Lös
KCl 15% 20ml Injektionslösung 2mmol/ml
Injektionskonzentrat
Metoclopramid Tropfen 0.3mg/Tr
Tropfen
Zolpidem Filmtabl 10mg
Filmtabl
Paracetamol Filmtabl 1000mg
Filmtabl
Haloperidol Tropfen
Tropfen
Folsäure Tabl 5mg
Tabl
Acetylsalicylsäure Filmtabl 100mg
Filmtabl
Amiodaron Tabl 200mg
Tabl
Pipamperon Tabl 40mg
Tabl
Esomeprazol Tabl 40mg
Tabl
Kalium chlorid Ret Drag
Ret Drag
Magnesium Gran 10mmol
Gran
Olmesartanmedoxomil Filmtabl 20mg
Filmtabl
Tamsulosin Ret Tabl 0.4mg
Ret Tabl
Prednison Tabl 50mg
Tabl
Risperi

In [291]:
import glob
import os

for file in glob.glob("*.txt"):
    annotated_file = os.path.splitext(file)[0] + '_annotated.xml'
    annotate_drugs(file, annotated_file)